In [1]:
import os
import openai
import weave
weave.init('together-weave')


Logged in as Weights & Biases user: swarajraibagi.
View Weave data at https://wandb.ai/swarajraibagi/together-weave/weave
🍩 https://wandb.ai/swarajraibagi/together-weave/r/call/0192171f-a47e-7cc1-93f9-866f8bdcf13d
Model response:
  San Francisco is a vibrant and diverse city located on the northern tip of the San Francisco Peninsula in California, United States. Known for its iconic Golden Gate Bridge, the city is a melting pot of cultures, with a rich history and a modern, innovative spirit.

The city's population is approximately 883,305 people, making it the fourth most populous city in California. San Francisco is famous for its steep hills, which are a defining feature of the city's landscape. The city's topography created by the surrounding mountains has made it a natural fortress, which historically protected it from invasions.

San Francisco boasts a substantial number of attractions and landmarks that make it an attractive destination for both tourists and locals. The most fam

In [ ]:


system_content = "You are a travel agent. Be descriptive and helpful."
user_content = "Tell me about San Francisco"

client = openai.OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
)
chat_completion = client.chat.completions.create(
    model="microsoft/phi-3-mini-128k-instruct:free",
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content},
    ],
    temperature=0.7,
    max_tokens=1024,
)
response = chat_completion.choices[0].message.content
print("Model response:\n", response)

In [2]:
def make_open_ai_request(model, messages, temperature=0.7, max_tokens=1024):
    client = openai.OpenAI(
        api_key=os.getenv("OPENROUTER_API_KEY"),
        base_url=os.getenv("OPENROUTER_BASE_URL"),
    )
    for attempt in range(3):
        try:
            chat_completion = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=temperature,
                max_tokens=max_tokens,
            )
            return chat_completion
        except Exception as e:
            if attempt == 2:  # Last attempt
                raise
            print(f"Attempt {attempt + 1} failed. Retrying...")
    raise Exception("Failed to make OpenAI request after 3 attempts")

make_open_ai_request(
    model='openai/chatgpt-4o-latest',
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content},
    ],
)

🍩 https://wandb.ai/swarajraibagi/together-weave/r/call/0192171e-5419-7ba3-91df-920024e18a87


ChatCompletion(id='gen-RJeYRHKoRd4JoscMXkbCiuokPjsH', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content="San Francisco is a vibrant and diverse city located in Northern California, known for its iconic landmarks, rich history, and dynamic culture. It’s a city that packs a lot into its compact 49 square miles, offering something for every type of traveler. Let me walk you through some of its highlights:\n\n### **1. Iconic Landmarks:**\n- **Golden Gate Bridge**: This world-famous suspension bridge, with its striking International Orange color, is one of the most photographed landmarks in the world. You can walk or bike across it for stunning views of the bay and the city skyline.\n- **Alcatraz Island**: Once a notorious federal prison, Alcatraz is now a popular tourist destination. A ferry ride takes you to the island where you can explore the prison and learn about its infamous inmates, including Al Capone. The audio tour is highly rec

In [2]:
from src.evaluation_suite_setup import EvaluationSuiteSetupConfig, SetupExample


example_setup = EvaluationSuiteSetupConfig(
    system_prompt="""You are a customer support agent tasked with writing the first response to a user filing a support issue. Follow these guidelines:

For content, begin with a warm greeting and thank the user for reaching out and acknowledge the issue they've reported.

For format, limit the response to 1 short paragraph.

For tone, maintain a professional yet friendly tone throughout.

Use the user's name if provided in their inquiry.

For additional rules, do not provide specific solutions in this first response.
Always end with an open-ended question to encourage further dialogue.
""",
    examples=[
        SetupExample(
            input="My shipment is delayed and I need it urgently.",
            output="Hello [User], Thank you for contacting our support team regarding your delayed shipment. I understand that this delay is causing you concern, especially given the urgency of your need. I sincerely apologize for any inconvenience this may be causing you. I want to assure you that I'm here to help and will do my best to address this situation as quickly as possible. Your satisfaction is important to us, and we take shipping delays very seriously. To better assist you, could you please provide me with your order number and the expected delivery date that was originally given to you? This information will help me investigate the status of your shipment more effectively."),
    ]
)


In [3]:
from src.parse_setup import parse_setup

parsed_setup = parse_setup(example_setup)
print(parsed_setup)

🍩 https://wandb.ai/swarajraibagi/together-weave/r/call/0192171f-e9f2-7481-8c1b-8e58dfd03a91
suite_description='The task is to generate an initial response to a customer support inquiry. The response must include a warm greeting, acknowledgment of the issue, and an open-ended question for further engagement. The response should be professionally friendly, concise, and should not offer specific solutions in this initial message.' evaluation_criteria=[EvaluationCriteria(criterion='Warm Greeting', explaination='The response should start with a warm greeting to make the customer feel valued and acknowledged.', evaluation_method='llm'), EvaluationCriteria(criterion='Acknowledgement of the Issue', explaination="Clearly acknowledging the customer's issue shows empathy and assures the customer that their concern is being taken seriously.", evaluation_method='llm'), EvaluationCriteria(criterion='Professional and Friendly Tone', explaination='A professional yet friendly tone is essential to ensur

{'suite_description': 'The task is to generate an initial response to a customer support inquiry. The response must include a warm greeting, acknowledgment of the issue, and an open-ended question for further engagement. The response should be professionally friendly, concise, and should not offer specific solutions in this initial message.',
 'evaluation_criteria': [{'criterion': 'Warm Greeting',
   'explaination': 'The response should start with a warm greeting to make the customer feel valued and acknowledged.',
   'evaluation_method': 'llm'},
  {'criterion': 'Acknowledgement of the Issue',
   'explaination': "Clearly acknowledging the customer's issue shows empathy and assures the customer that their concern is being taken seriously.",
   'evaluation_method': 'llm'},
  {'criterion': 'Professional and Friendly Tone',
   'explaination': 'A professional yet friendly tone is essential to ensure the customer feels comfortable and respected.',
   'evaluation_method': 'llm'},
  {'criterio